In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from pathlib import Path


In [30]:
data_path = Path("../data/processed/feature_data.csv")
df=pd.read_csv(data_path)


In [31]:
cat_cols = [col for col in df.columns if df[col].nunique() < 20 and df[col].dtype in ['int64', 'object']]


In [32]:
from sklearn.model_selection import train_test_split

# Assume you already have your feature matrix X and target y
X = df.drop("readmitted", axis=1)
y = df["readmitted"]

# Split into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [33]:
x_train.columns = [col.replace('[', '_').replace(']', '_').replace('<', '_') for col in x_train.columns]
x_test.columns = [col.replace('[', '_').replace(']', '_').replace('<', '_') for col in x_test.columns]

In [34]:
# Step 1: Clean column names (if not already done)
X_train.columns = [col.replace('[', '_').replace(']', '_').replace('<', '_') for col in X_train.columns]
X_valid.columns = [col.replace('[', '_').replace(']', '_').replace('<', '_') for col in X_valid.columns]

# Step 2: Import required modules
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from scipy.stats import randint, uniform
import xgboost as xgb

# Step 3: Define base learners
estimators = [
    ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)),
    ('rf', RandomForestClassifier(random_state=42))
]

# Step 4: Build stacking model
stack = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(max_iter=1000),
    cv=5,
    n_jobs=1  # Only 1 thread to avoid memory error
)

# Step 5: Define parameter search space
param_dist = {
    'xgb__max_depth': randint(3, 7),
    'xgb__learning_rate': uniform(0.01, 0.1),
    'xgb__n_estimators': randint(100, 200),
    'xgb__subsample': uniform(0.6, 0.4),
    'xgb__colsample_bytree': uniform(0.6, 0.4),
    'xgb__gamma': uniform(0, 1),

    'rf__n_estimators': randint(100, 200),
    'rf__max_depth': [None, 5, 10],
    'rf__max_features': ['sqrt', 'log2'],

    'final_estimator__C': uniform(0.1, 5.0)
}

# Step 6: Run randomized search (memory-safe version)
random_search = RandomizedSearchCV(
    estimator=stack,
    param_distributions=param_dist,
    n_iter=20,              # Reduced from 50 for speed & memory
    scoring='roc_auc',
    cv=3,                   # Reduced CV folds for speed
    verbose=2,
    n_jobs=1,               # ✅ This prevents memory crash
    random_state=42
)

# Step 7: Fit the model
random_search.fit(X_train, y_train)

# Step 8: Evaluate on validation set
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_valid)[:, 1]
auc = roc_auc_score(y_valid, y_pred_proba)

print("✅ Tuned Stacked AUC:", auc)


Fitting 3 folds for each of 20 candidates, totalling 60 fits


d:\hospital\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:13:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\hospital\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:13:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\hospital\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:13:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\hospital\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:13:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\h

[CV] END final_estimator__C=1.9727005942368125, rf__max_depth=None, rf__max_features=sqrt, rf__n_estimators=171, xgb__colsample_bytree=0.8394633936788146, xgb__gamma=0.15601864044243652, xgb__learning_rate=0.025599452033620268, xgb__max_depth=5, xgb__n_estimators=174, xgb__subsample=0.9464704583099741; total time= 1.6min


d:\hospital\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:14:52] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\hospital\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:15:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\hospital\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:15:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\hospital\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:15:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\h

[CV] END final_estimator__C=1.9727005942368125, rf__max_depth=None, rf__max_features=sqrt, rf__n_estimators=171, xgb__colsample_bytree=0.8394633936788146, xgb__gamma=0.15601864044243652, xgb__learning_rate=0.025599452033620268, xgb__max_depth=5, xgb__n_estimators=174, xgb__subsample=0.9464704583099741; total time= 1.7min


...

[CV] END final_estimator__C=0.8244743604561154, rf__max_depth=5, rf__max_features=log2, rf__n_estimators=169, xgb__colsample_bytree=0.6968221086046001, xgb__gamma=0.6721355474058786, xgb__learning_rate=0.08616196153287176, xgb__max_depth=5, xgb__n_estimators=116, xgb__subsample=0.8912865394447438; total time=  11.1s


d:\hospital\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:53:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\hospital\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:53:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\hospital\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:53:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\hospital\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:53:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\h

[CV] END final_estimator__C=0.8244743604561154, rf__max_depth=5, rf__max_features=log2, rf__n_estimators=169, xgb__colsample_bytree=0.6968221086046001, xgb__gamma=0.6721355474058786, xgb__learning_rate=0.08616196153287176, xgb__max_depth=5, xgb__n_estimators=116, xgb__subsample=0.8912865394447438; total time=  11.3s


d:\hospital\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:53:53] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\hospital\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:53:59] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\hospital\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:54:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\hospital\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:54:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\h

✅ Tuned Stacked AUC: 0.6691485655990823


In [ ]:
mod_params = {
    'rf': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_features': ['auto', 'sqrt'],
            'max_depth': [10, 30, 50],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'bootstrap': [True, False]
        }
    },
    'xgb': {
        'model': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        'params': {
            'n_estimators': [50, 100, 200, 300],
            'learning_rate': [0.01, 0.05, 0.1, 0.2],
            'max_depth': [3, 5, 7, 10],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0],
            'gamma': [0, 0.1, 0.3, 0.5]
        }
    },
    'lgbm': {
        'model': LGBMClassifier(),
        'params': {
            'n_estimators': [100, 200, 300, 400],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 5, 7, 10, -1],
            'num_leaves': [15, 31, 50, 70],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0]
        }
    }
}